In [11]:
import numpy as np
import pandas as pd
from datetime import datetime, date, time
from tqdm import tqdm

In [12]:
#_SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202311061246.csv
CSV_filename = input("Enter the CSV filename for the EA history: ")

print("Extracting script history...")

In [ ]:
sql_extract = pd.read_csv("./data/execution/" + CSV_filename, low_memory=False)

sql_extract.rename(columns={'linenum': 'lineNum', 'executionid': 'executionLogsId', 'scriptname': 'scriptName', 'label': 'label', 'status': 'step_status', 'starttime': 'startTime', 'endtime': 'endTime', 'duration': 'duration', 'id': 'StepLogsId'}, inplace=True)

In [13]:
history_length = len(sql_extract)
print(f"{history_length} records imported from the EA history file {CSV_filename}")

5478863 records imported from the EA history file _SELECT_ExecutionLog_scriptname_ExecutionStepLog_id_ExecutionSte_202311061246.csv


In [14]:
# rundate = '2023-11-03'
# run_start_time = 8.0 # Time is UTC
# run_finish_time = 23.0

rundate = input("Enter the date of the test run as yyyy-mm-dd: ")
run_start_time = input("Enter the start time of the run in the format h.mm (UTC time, no leading zeros): ")
run_finish_time = input("Enter the finish time of the run in the format h.mm (UTC time, no leading zeros): ")
run_start_time = float(run_start_time)
run_finish_time = float(run_finish_time)


In [15]:
print(f"Extracting test results generated on {rundate}, between {run_start_time} and {run_finish_time}.")

Extracting for test results generated on 2023-11-03, between 8.0 and 23.0.


In [16]:
df = sql_extract.copy()
df.rename(columns={'label':'StepLabel'}, inplace=True)
df = df.dropna(subset = ['scriptName']).copy()

In [17]:

df = df.dropna(subset=['startTime']).copy()
df = df.dropna(subset=['endTime']).copy()
df = df.sort_values(by=['startTime'], ascending=False).copy()
df = df.reset_index(drop=True)

In [18]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [19]:
df['duration_sec'] = df.duration/1000
df['duration_mins'] = df.duration/1000/60

In [20]:
#df['date'] = pd.to_datetime(df['startTime'], format='%Y-%m-%d', utc=True).dt.date
df['date'] = pd.to_datetime(df['startTime'], format='mixed', utc=True).dt.date
df['date'] = pd.to_datetime(df['date'])


In [21]:
df = df.query("date == @rundate").copy()  # Remember time is UTC

In [22]:
df['time'] = pd.to_datetime(df['startTime'], utc=True).dt.time

In [23]:
df['time_dec'] = df['time'].astype(str)
df['time_dec'] = df['time_dec'].str[:5]
df['time_dec'] = df['time_dec'].str.replace(':','.')
df['time_dec'] = df['time_dec'].astype(float)

### Add a column to indicate Runs of each test script
As there is no other field available via the API, this field is generated each time the lineNum == 1.  This assumes that there will always be a step 1 in every test case.
Before this can be done, first sort the whole dataframe

In [24]:
df.sort_values(by=['executionLogsId','StepLogsId'], inplace=True)
df = df.reset_index(drop=True)

In [25]:
# This is the new code to allocate a 'run' number and increment it every time the lineNum == 1.  This was generated by ChatGPT based on the code above :-)

df['run'] = (df['lineNum'] == 1).cumsum()

### Generate a field to indicate where test scripts have run to the last step of the script
This assumes that the last step is "End script"
First sort the dataframe by Runs
Then add a new text field that is a concatenation of all the step labels for each run
Finally, check which of those strings contain the text "End script"

In [26]:
runs_np = df.run.unique()
end_script_dict = {}
for i in runs_np:
    end_script_dict[i] = 'Script stopped'
step_error_dict = {}
for i in runs_np:
    step_error_dict[i] = 'No step errors'

In [27]:
df.sort_values(by=['run'], inplace=True)
df = df.reset_index(drop=True)

In [28]:
for idx in df.index:
    #if run == 10: break
    run = df['run'][idx]
    if df['StepLabel'][idx] == 'End script':
        end_script_dict[run] = 'Script completed'
    if df['step_status'][idx] == 'ERROR':
        step_error_dict[run] = 'One or more step errors'

In [29]:
end_script_df = pd.DataFrame.from_dict(end_script_dict,orient ='index',columns=['script_completion'])
end_script_df.reset_index(inplace=True)
end_script_df.rename(columns={'index':'run'}, inplace=True)

In [30]:
step_error_df = pd.DataFrame.from_dict(step_error_dict,orient ='index',columns=['step_error'])
step_error_df.reset_index(inplace=True)
step_error_df.rename(columns={'index':'run'}, inplace=True)

In [31]:
df = pd.merge(
    df,
    end_script_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [32]:
df = pd.merge(
    df,
    step_error_df,
    how="inner",
    on=None,
    left_on='run',
    right_on='run',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [33]:
df.rename(columns={'script_status': 'EA_script_status','step_state': 'EA_step_state','step_status': 'EA_step_status','step_error': 'IPT_step_error','script_completion': 'IPT_script_completion'}, inplace=True)

In [34]:
df['KPI'] = 'Not mapped'

In [35]:
# K1
mask = (df['scriptName'].str.contains('Place a sales local order',case=False))
df['KPI'].mask(mask,'K01 Place a sales local order',inplace=True)

# K2
mask = (df['scriptName'].str.contains('Place a B2B local order',case=False))
df['KPI'].mask(mask,'K02 Place a B2B/EDI order and Send Confirmation',inplace=True)

# K3 Place Service Portal Order
# No test script

# K4
mask = (df['scriptName'].str.contains('Amend an existing sales order',case=False))
df['KPI'].mask(mask,'K04 Amend an existing sales order',inplace=True)

# K5
mask = (df['scriptName'].str.contains('Trade Returns Order',case=False))
df['KPI'].mask(mask,'K05 Place a Trade Returns Order',inplace=True)

# K6
mask = (df['scriptName'].str.contains('Submit a B2B Remittance',case=False))
df['KPI'].mask(mask,'K06 B2B Remittance',inplace=True)

# K7 B2B Remittance Adjustment
# No test script

#K8
mask = (df['scriptName'].str.contains('Receipted payment processing',case=False))
df['KPI'].mask(mask,'K08 Receipted payment processing SA Only at the moment',inplace=True)

# K9 aDSD Batch Job posting invoice for HHD billing document 
# No test script

# K10
mask = (df['scriptName'].str.contains('Credit status check',case=False))
df['KPI'].mask(mask,'K10 Credit status check',inplace=True)

# K11
mask = (df['scriptName'].str.contains('Send Load to Roadnet',case=False))
df['KPI'].mask(mask,'K11 Send Load to Roadnet load for Planning',inplace=True)

# K12
mask = (df['scriptName'].str.contains('Receive load from Roadnet',case=False))
df['KPI'].mask(mask,'K12 Receive load from Roadnet into D365',inplace=True)

# K13b
mask = (df['scriptName'].str.contains('Release to warehouse',case=False) & df['scriptName'].str.contains('Roadnet loads',case=False))
df['KPI'].mask(mask,'K13b Release to Warehouse & Complete Picking Work (Roadnet loads)',inplace=True)
# Release to Warehouse (Roadnet loads) & Complete Picking Work & Process OOS

# K13
mask = (df['scriptName'].str.contains('Release to Warehouse',case=False) & df['scriptName'].str.contains('manual',case=False))
df['KPI'].mask(mask,'K13 Release to Warehouse (manually planned loads)',inplace=True)

# K14 Complete Picking Work & Process OOS 
# Not mapped??

# K14b
mask = (df['scriptName'].str.contains('Complete Picking Work',case=False) & df['scriptName'].str.contains('manual loads',case=False))
df['KPI'].mask(mask,'K14b Complete Picking Work & Process OOS (manually planned loads)',inplace=True)

# K15
mask = (df['scriptName'].str.contains('aDSD Load confirmation',case=False))
df['KPI'].mask(mask,'K15 aDSD Load confirmation',inplace=True)

# K15 (UG1, MZ1)
mask = (df['scriptName'].str.contains('Invoice Load confirmation',case=False))
df['KPI'].mask(mask,'K15 Invoice Load confirmation',inplace=True)

# K16
mask = (df['scriptName'].str.contains('Load upload',case=False) & df['scriptName'].str.contains('Settlement',case=False))
df['KPI'].mask(mask,'K16 Load upload & Settlement',inplace=True)

# K17
mask = (df['scriptName'].str.contains('Create a cost estimate',case=False))
df['KPI'].mask(mask,'K17 Create a cost estimate for all standard costed procured materials',inplace=True)

# K18
mask = (df['scriptName'].str.contains('Imported Statistical',case=False))
df['KPI'].mask(mask,'K18 Import Actual Statistical entries, into Cost Accounting',inplace=True)

# K19
mask = (df['scriptName'].str.contains('Distribute the range of items pending prices',case=False))
df['KPI'].mask(mask,'K19 Distribute the range of items pending prices to another site',inplace=True)

# K20
mask = ((df['scriptName'].str.contains('Cost Rollup',case=False) & df['scriptName'].str.contains('Cost Allocation',case=False)) | df['scriptName'].str.contains('Maintain cost distribution table',case=False))
df['KPI'].mask(mask,'K20 Run Cost Rollup and Cost Allocation Policies',inplace=True)

# K21
mask = (df['scriptName'].str.contains('Place SFA order',case=False))
df['KPI'].mask(mask,'K21 Place SFA order and Send Confirmation',inplace=True)

In [36]:
df.sort_values(by=['KPI', 'scriptName', 'executionLogsId', 'run', 'lineNum'], inplace=True)
df = df.reset_index(drop=True)

In [37]:
df = df.dropna(subset=['StepLabel']).copy()

### Filter the dataset for output on a specific test cycle

In [38]:
df1 = df.query("date == @rundate and time_dec > @run_start_time and time_dec < @run_finish_time").copy()  # Remember time is UTC

In [39]:
df1 = df1.reset_index(drop=True).copy()

### Mark those Test Script Steps that must be included in the time measurement

In [40]:
print("Generating KPIs")

Generating KPIs


In [41]:
df1['KPI_no'] = ''

In [42]:
df1['Include_in_measure'] = False

In [43]:
kpi_numbers = ['K11', 'K12', 'K13', 'K14', 'K15', 'K16', 'K17', 'K18', 'K19', 'K20',
               'NAK11', 'NAK12', 'NAK13', 'NAK14', 'NAK15', 'NAK16', 'NAK17', 'NAK18', 'NAK19', 'NAK20',
               'MZK11', 'MZK12', 'MZK13', 'MZK14', 'MZK15', 'MZK16', 'MZK17', 'MZK18', 'MZK19', 'MZK20',
               'UGK11', 'UGK12', 'UGK13', 'UGK14', 'UGK15', 'UGK16', 'UGK17', 'UGK18', 'UGK19', 'UGK20']


In [44]:
for idx in df1.index:
    label = df1['StepLabel'][idx]
    prefix = label[0:5]
    if prefix in kpi_numbers:
        if label[5] == 'b':
            df1.at[idx, 'KPI_no'] = prefix + 'b'
        else:
            df1.at[idx, 'KPI_no'] = prefix  
        df1.at[idx, 'Include_in_measure'] = True

for idx in df1.index:
    label = df1['StepLabel'][idx]
    prefix = label[0:3]
    if prefix in kpi_numbers:
        if label[3] == 'b':
            df1.at[idx, 'KPI_no'] = prefix + 'b'
        else:
            df1.at[idx, 'KPI_no'] = prefix  
        df1.at[idx, 'Include_in_measure'] = True        

 34%|███▍      | 20656/59908 [00:00<00:00, 206539.02it/s]

100%|██████████| 59908/59908 [00:00<00:00, 147636.88it/s]


In [45]:
filename = 'data/execution/' + rundate +  '_results_filtered.csv'
df1.to_csv(filename,index=False)

In [46]:
df2 = df1.query("Include_in_measure == True and EA_step_status != 'ERROR'").copy()

In [48]:
df2 = df2.drop_duplicates(keep='first')
df2 = df2.reset_index(drop=True)

In [49]:
# 90th Percentile
def q90(x):
    return x.quantile(0.9)


kpi_results = df2.groupby(['KPI_no']).agg({'KPI': 'first', 'duration_sec': ['mean', 'max', q90, 'std'], 'run': 'count'}).reset_index()

# Flatten multi-level columns
kpi_results.columns = ['_'.join(col) for col in kpi_results.columns]
kpi_results = kpi_results.reset_index(drop=True)

In [50]:
kpi_results.rename(columns={'KPI_no_': 'KPI_no', 'KPI_first':'KPI Process', 'duration_sec_mean': 'Average', 'duration_sec_max': 'Max', 'duration_sec_q90': '90th_%', 'duration_sec_std': 'SD', 'run_count': 'Successful Runs'}, inplace=True)

In [52]:
kpi_results = kpi_results.round(decimals=1)

In [54]:
kpi_results

,KPI_no,KPI Process,Average,Max,90th_%,SD,Successful Runs
0,K11,K11 Send Load to Roadnet load for Planning,3.7,7.5,5.1,1.1,120
1,K12,K12 Receive load from Roadnet into D365,187.9,356.0,317.9,120.7,7
2,K13,K13 Release to Warehouse (manually planned loads),17.8,40.3,33.8,12.1,8
3,K13b,K13b Release to Warehouse & Complete Picking W...,19.1,105.8,34.1,14.6,173
4,K14,K13 Release to Warehouse (manually planned loads),39.0,66.5,65.5,23.4,8
5,K14b,K13b Release to Warehouse & Complete Picking W...,21.8,133.9,54.6,22.8,404
6,K15,K15 Invoice Load confirmation,29.4,531.9,67.4,34.0,604
7,K18,"K18 Import Actual Statistical entries, into Co...",54.8,54.8,54.8,NaN,1
8,MZK11,K11 Send Load to Roadnet load for Planning,3.9,4.8,4.6,0.8,6
9,MZK13,K13 Release to Warehouse (manually planned loads),12.6,16.1,15.3,3.5,4


In [55]:
print("Generating output files")

filename = 'data/execution/' + rundate +  '_kpi_results.xlsx'
kpi_results.to_excel(filename,index=False)

Generating output files
